# Write a succinct summary of your observations for all the tasks
#### 2.1 Data-preparation
Of all tasks, data preparation required manual curation. The Bengali corpus needed extensive cleaning like stripping off English strings, emojis, country flag symbols, and repeated occurrence of the same punctuation. Also, the lines had to be segmented into sentences based on punctuation marks like `?`,`!`,`|`.

#### 2.3 Subword units
The goal of this task is to create data for language modeling based on different levels of subword granularity. Following observations were noted for each subword granularity:

1. With granularity on character level (vocabulary size=52), the generated text is segmented on the character level i.e each word being broken into characters. For example, the token `মা` (pronounced as `Mā`) is segmented as `_ম` and` া` where the ` া` character implies the `ā` sound.
2. With granularity set to smaller subwords units closer to characters (vocab size=400), the generated text is segmented between character level and word level. That is to say, the tokens are combinations of a few characters but not a complete word. For example, the word `পাওয়া` (pronounced as `Pā'ōẏā`) is segmented as `▁পা` an `ওয়া`. As can be seen, both tokens are a combination of 2 characters.
3. With granularity set to larger subwords units closer to words (vocab size=3000), the generated text is segmented precisely on word level with some occurrences of segmentation on character level. For example: `স্বার্থ` segmented into `▁সার ্থ` (pronounced as `Sbārtha`) 

#### 3. LM training
In this task, we train the RNN language model based on different subword granularity. For baseline we set the number of hidden layers (`-hid`) as 40 and amount of steps to propagate error (`-bptt`) to 3. It is observed that perplexity decreases with every iteration of training and the model with the best perplexity score is saved. This model is further used at a  later stage for text generation. Not to mention, training time increased with an increase in vocabulary size.
<br>
**TODO: Discuss hyper-parameter tuning in further section**

#### 4. Text generation
In this task, we use the trained RNN based language model to create artificial data. Following observations were made for each subword granularity:
1. Level 1 granularity: The generated text is entirely meaningless and grammatically incorrect. The words in themselves are inexistent and carry no meaning. Moreover, the words are quite short i.e. resemble characters as it was generated on character level granularity.
2. Level 2 granularity: There seem very few real and meaningful words, but the generated text as in whole carries no meaning. The text is slightly longer than level 1 granularity.
3. Level 3 granularity: Text generated on subword units closer to words seems to be the best of the above, although is partially meaningful. It has a more meaningful and rich vocabulary than level 2 granularity and the sentence structure is partially correct.

Also, it was quite interesting to observe text generated after hyperparameter tuning. Improved sentence structure with more meaningful words were found in the text that resulted from best hyperparameters with improved OOV rate and lower perplexity than the baseline.

#### 5. OOV comparison
In this task, we compute the out-of-vocabulary (OOV) rate on the given corpus (using the train and test split that we performed earlier) and compare it with the OOV rate computed after augmenting the training vocabulary. In other words, we use vocabulary from the artificially generated text and add to the training vocabulary. 

It is observed that as the size of the generated corpus increases i.e as $k$ increases, the OOV rate decreases. With the model generating more text for each $k$, it covers up most of the words from the test set, which explains the decrease in the OOV rate. The least OOV rate is observed for $k=7$, however, the generated text is quite long.
For practical applications, a model with lower perplexity and lower OOV and at the same time gives meaningful sentences i.e. model with smaller subword vocabulary would be a preferable option.

**Comparison of Perplexity and OOV rate vs Vocabulary size.**
It is observed that with an increase in vocabulary size, perplexity increases i.e. the language model deteriorates, and the OOV rate decreases. Below results were observed for `hid=40` and `bptt=3`

| Vocab Size |   PPL   | OOV |
|------------|:-------:|-----------|
|         52 |  10.026 |    0.1353 |
|         80 |  16.489 |    0.1347 |
|        100 |  21.925 |    0.1331 |
|        200 |  41.832 |    0.1301 |
|        400 |  62.795 |    0.1248 |
|        800 | 124.997 |    0.1203 |
|       2000 | 278.289 |    0.1154 |
|       2500 | 326.707 |    0.1147 |
|       3000 | 376.060 |    0.1143 |
|       4000 | 448.427 |    0.1127 |

## what you aimed to achieve
Throughout this project we aimed to use subword representation to estimate OOV words. To achieve this we trained a RNN based language model to artifically generate corpus and compute OOV rate on varying sizes of the generated corpora. Our aim was to achieve best OOV rate on the generated corpora by playing around with different hyperparameters of RNNLM while also maintaining a below baseline perplexity score.

## whether your expectations were fulfilled.
We managed to achieve a better OOV rate and perplexity score than the baseline for all three levels of granularity with appropriate hyperparameter tuning (specifics to be discussed in further section). We also observed that with increase in vocabulary size, the perplexity esclated, while OOV rate dipped slowly.

## What are your takeaways from this project


## How do your results differ for English and Bengali?
1. For larger vocabulary Bengali was found to have much higher perplexity than English. This may be due the fact Bengali is a morphologically richer language than English.

2. We also observed that while Bengali showed uniform increase in perplexity and decrease in OOV rate with increasing vocab size. However this behaviour for English was not consistent. Net perplexity decreased by 8.47 when vocab size increased from 1600 to 2000. While OOV rate was expected to decrease with increasing vocabulary size, it esclated continiously from 0.0619 to 0.0797 while increasing vocabulary size from 650 to 2500.

3. For all granularity levels, English had much lower OOV rate than Bengali. This is again can be explained as Bengali is a morphologically rich language.  

## What hyperparameters do you use to optimise the OOV rates?
We observed that increasing number of hidden layer and amount of steps to propagate error (bptt), the perplexity declined as well as OOV improved.

**Parameters table**

| Params\Models | Baseline (fixed) | Character | Smaller Vocab | Larger Vocab |
|---------------|------------------|-----------|---------------|--------------|
| hidden        |               40 |        120|           120 |           120|
| bptt          |                3 |         4 |             4 |            4 |
| class         | #vocab_size      |        52 |           400 |         3000 |


**Perplexity scores**

|Model\ Granularity|	Characters	| Smaller vocabulary 	| Larger vocabulary |
|------------      |          ------|---                    |---                |
|Baseline          |	   10.026335|	           62.794973|	      376.060040|
|Custom	           |        7.311287|              49.141582|	      365.665047|

**OOV rate** <br>
OOV on given corpus: 0.157

|Model\ Granularity|	Characters	   |    Smaller vocabulary |      Larger vocabulary|
|------------      |------             |---                    |                   --- |
|Baseline	       |0.1352             |	             0.1248|	             0.1143|
|Custom	           |0.1297             |                 0.1202|                 0.1133|

As can be concluded from the table above, the hyper-parameters that outperform the baseline results are 120 hidden layers and 4 bptt. It must also be mentioned that with 120 hidden layers, the training time was quite significant. 

We also had a chance to experiment with lower hidden layers and bptt, for which the OOV rates were less than those of baseline. However, we believed increasing the hidden layers and bptt would be a better choice for achieving an improved OOV rate and lower perplexity.

A slightly better result may be achieved with more hidden layers and more steps to propagate error, but it would come at a cost of an increased training time which we believe would not be a suitable choice for practical purposes.

## Are there any ways you could improve your results
Here are few ways we could improve the results:
1. We could try training rnnlm with more hidden layers and bptt, but at a cost of increased training complexity.
2. We can also search for optimal vocabulary sizes extensively. Some techniques like grid search with small changes of values could give improvement in model performance.
3. A better neural network architectures like transformers can also improve the results.

Anymore ideas?